# Communicate with Hyperliquid API and check the OrderBook (price, spread, VWAP, average buying price)
 for details refer to https://github.com/ccxt/ccxt/wiki/Manual#overview
 no need to api keys here so all good

In [5]:
import ccxt
print(ccxt.exchanges)

['ace', 'alpaca', 'ascendex', 'bequant', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bingx', 'bit2c', 'bitbank', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitteam', 'bitvavo', 'bl3p', 'blockchaincom', 'blofin', 'btcalpha', 'btcbox', 'btcmarkets', 'btcturk', 'bybit', 'cex', 'coinbase', 'coinbaseadvanced', 'coinbaseexchange', 'coinbaseinternational', 'coincatch', 'coincheck', 'coinex', 'coinlist', 'coinmate', 'coinmetro', 'coinone', 'coinsph', 'coinspot', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'ellipx', 'exmo', 'fmfwio', 'gate', 'gateio', 'gemini', 'hashkey', 'hitbtc', 'hollaex', 'htx', 'huobi', 'huobijp', 'hyperliquid', 'idex', 'independentreserve', 'indodax', 'kraken', 'krakenfutures', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'luno', 'lykke', 'mercado', 'mexc', 'ndax', 'novadax', 'oceanex', 'okcoin', 'okx', 'onetra

In [6]:
exchange = ccxt.hyperliquid()
exchange

ccxt.hyperliquid()

In [ ]:
markets = exchange.load_markets()
print(exchange.id, markets)

In [ ]:
exchange.fetch_currencies()

In [ ]:
exchange.fetch_markets()

In [10]:
OrderBookBTC = exchange.fetchOrderBook('BTC/USDC:USDC')

In [11]:
OrderBookBTC.keys()

dict_keys(['symbol', 'bids', 'asks', 'timestamp', 'datetime', 'nonce'])

In [13]:
import pandas as pd

ob = pd.DataFrame(OrderBookBTC)

print(ob)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
           symbol                 bids                asks      timestamp  \
0   BTC/USDC:USDC   [93696.0, 0.23452]     [93697.0, 0.45]  1732704761072   
1   BTC/USDC:USDC     [93695.0, 0.625]  [93698.0, 0.88066]  1732704761072   
2   BTC/USDC:USDC    [93693.0, 0.3125]  [93699.0, 0.25115]  1732704761072   
3   BTC/USDC:USDC    [93692.0, 0.3125]  [93700.0, 0.18357]  1732704761072   
4   BTC/USDC:USDC   [93691.0, 0.13332]  [93701.0, 0.72348]  1732704761072   
5   BTC/USDC:USDC   [93690.0, 0.14186]

Lets look at the price a bit, what is the mid price?

In [14]:
best_bid = ob['bids'][0][0]  # Best bid price (highest) - price you gotta pay to buy
best_ask = ob['asks'][0][0]  # Best ask price (lowest) - price you receive if you sell

mid_price = (best_bid + best_ask) / 2
print("best bid", best_bid)
print('best ask', best_ask)
print(f"Mid-Price: {mid_price}")

best bid 93696.0
best ask 93697.0
Mid-Price: 93696.5


In [15]:
print('spread', best_ask - best_bid)

spread 1.0


What if I make a big order, how is gonna be the price?

In [16]:
def price_for_order(side, size, orderbook):
    bookside = orderbook[side]
    
    tot_quantity = 0
    cost = 0
    
    for price, available_quantity in bookside:
        #print('available_quantity', available_quantity)
        #print('price', price)
        #print('tot executed', tot_quantity)
        if tot_quantity + available_quantity >= size:
            cost += price*(size-tot_quantity)
            tot_quantity += size-tot_quantity
            break
        else:
            cost += price*(available_quantity)
            tot_quantity += available_quantity
             
    return cost if tot_quantity == size else 'error'

def vwap_for_orderbook(side, orderbook):
    bookside = orderbook[side]
    
    tot_quantity = 0
    cost = 0
    
    for price, available_quantity in bookside:
        cost += price*(available_quantity)
        tot_quantity += available_quantity
             
    return cost/tot_quantity


quantity = 20
side = 'bids'

price = price_for_order( side, quantity, ob)
vwap = vwap_for_orderbook( side, ob)
print('\n')
print(f"Price for {quantity} units: {price}")
print('compared to ideal market price')
print(f"Mid-Price: {mid_price*quantity}")
print('\n')
print(f"VWAP for the orderbook {side}: {vwap}")
print('compared to ideal average market price')
print(f"Mid-Price: {mid_price}")
            



Price for 20 units: 1873619.6148599999
compared to ideal market price
Mid-Price: 1873930.0


VWAP for the orderbook bids: 93679.35735264434
compared to ideal average market price
Mid-Price: 93696.5
